In [13]:
import mlflow
import mlflow.xgboost
from mlflow.models.signature import infer_signature
from joblib import load
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set MLflow tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Set experiment name
EXPERIMENT_NAME = "Sales Forecasting and Optimization"
mlflow.set_experiment(EXPERIMENT_NAME)

def log_model_metrics(model, X_test, y_test, model_name="xgb_model"):
    """Log model metrics and artifacts to MLflow"""
    with mlflow.start_run(run_name=f"Evaluation of {model_name}"):
        # Make predictions
        predictions = model.predict(X_test)
        
        # Calculate metrics
        rmse = mean_squared_error(y_test, predictions)
        mae = mean_absolute_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        
        # Log metrics
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        
        # Log model parameters
        mlflow.log_params(model.get_params())
        
        # Infer signature
        signature = infer_signature(X_test, predictions)
        
        # Log the model
        mlflow.xgboost.log_model(
            model,
            "model",
            signature=signature,
            registered_model_name=model_name
        )
        
        # Log model info
        mlflow.log_text("Model evaluation metrics and parameters", "model_info.txt")
        
        print(f"Logged metrics - RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.2f}")

# Load the model and data
model = load(r'Saved Models\xgb_model.joblib')
scaler = load(r'Saved Models\scaler.joblib')
data = pd.read_csv(r'walmart_cleaned_machine.csv')

features = ['Fuel_Price', 'Temperature', 'CPI', 'Unemployment','Size', 'Weekly_Sales']

data[features] = scaler.transform(data[features])

X = data.drop(columns=['date', 'Weekly_Sales'])
y = data['Weekly_Sales']


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Log model metrics
log_model_metrics(model, X_test, y_test)

c:\Users\mahmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\mahmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [16:38:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the

Logged metrics - RMSE: 0.00, MAE: 0.02, R2: 1.00
🏃 View run Evaluation of xgb_model at: http://127.0.0.1:5000/#/experiments/793749244390985861/runs/73a377275af04b2a976066b0e8e8cc3d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/793749244390985861


Created version '4' of model 'xgb_model'.
